# Lab 4: Transfer Attacks

## Overview

Transfer attacks exploit the phenomenon that adversarial examples crafted for one model often fool other models. This is crucial when you can't query the target model directly.

## Learning Objectives

1. Understand adversarial transferability
2. Generate transferable adversarial examples
3. Test cross-model attacks
4. Use ensemble methods for better transfer
5. Evaluate transfer success rates

## Prerequisites

- Completion of Labs 1-3
- Understanding of gradient-based attacks

In [1]:
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms, models
import numpy as np
import matplotlib.pyplot as plt

# Detect device (supports CUDA, Apple Silicon MPS, and CPU)
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"✓ Using CUDA GPU: {torch.cuda.get_device_name(0)}")
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = torch.device('mps')
    print("✓ Using Apple Silicon GPU (MPS)")
else:
    device = torch.device('cpu')
    print("ℹ Using CPU")

print(f"Device: {device}")

✓ Using Apple Silicon GPU (MPS)
Device: mps


## Setup: Load Multiple Models

In [2]:
# Load multiple models for transfer testing
models_dict = {
    'ResNet18': models.resnet18(pretrained=True),
    'ResNet50': models.resnet50(pretrained=True),
    'VGG16': models.vgg16(pretrained=True),
    'MobileNetV2': models.mobilenet_v2(pretrained=True)
}

# Set all to eval mode
for name, model in models_dict.items():
    model.eval()
    model.to(device)
    print(f'✓ Loaded {name}')

# Preprocessing
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

/Users/schwartz/src/genai-security-training/.venv/lib/python3.14/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/schwartz/src/genai-security-training/.venv/lib/python3.14/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/Users/schwartz/src/genai-security-training/.venv/lib/python3.14/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /Users/schwartz/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


100%|██████████| 97.8M/97.8M [00:59<00:00, 1.72MB/s]
/Users/schwartz/src/genai-security-training/.venv/lib/python3.14/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /Users/schwartz/.cache/torch/hub/checkpoints/vgg16-397923af.pth


100%|██████████| 528M/528M [05:29<00:00, 1.68MB/s] 
/Users/schwartz/src/genai-security-training/.venv/lib/python3.14/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


✓ Loaded ResNet18
✓ Loaded ResNet50
✓ Loaded VGG16
✓ Loaded MobileNetV2
